In [0]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.window import Window

consumidores_bronze_df = spark.table("medalhao.bronze.ft_consumidores")
pedidos_bronze_df = spark.table("medalhao.bronze.ft_pedidos")
pedidos_itens_bronze_df = spark.table("medalhao.bronze.ft_itens_pedidos")
pagamentos_bronze_df = spark.table("medalhao.bronze.ft_pagamentos_pedidos")
produtos_bronze_df = spark.table("medalhao.bronze.ft_produtos")
categoria_produtos_traducao_bronze_df = spark.table("medalhao.bronze.dm_categoria_produtos_traducao")
vendedores_bronze_df = spark.table("medalhao.bronze.ft_vendedores")
avaliacoes_pedidos_bronze_df = spark.table("medalhao.bronze.ft_avaliacoes_pedidos")


In [0]:
consumidores_bronze_df.printSchema()

In [0]:
consumidores_bronze_df_count_duplicates = (consumidores_bronze_df.groupBy('customer_id').count().filter(F.col('count') > 1))


display(consumidores_bronze_df.limit(10))

In [0]:
consumidores_bronze_df.printSchema()

In [0]:
consumidores_silver_df = (consumidores_bronze_df
                          .select(
                              F.col('customer_id').alias('id_consumidor'),
                              F.upper(F.col('customer_city')).alias('cidade'),
                              F.col('customer_zip_code_prefix').alias('prefixo_cep'),
                              F.upper(F.col('customer_state')).alias('estado'),
                              F.col('ingestion_timestamp').alias('data_ingestao')
                          )
                          .dropDuplicates(['id_consumidor'])
                          .withColumn('data_ingestao', F.current_timestamp()))
display(consumidores_silver_df.limit(20))

In [0]:
pedidos_bronze_df.printSchema()

pedidos_bronze_df = (pedidos_bronze_df.
                     withColumn('order_status', F.when(F.col('order_status') == 'canceled', 'cancelado').
                                            when(F.col('order_status') == 'shipped', 'enviado').
                                            when(F.col('order_status') == 'processing', 'em processamento').
                                            when(F.col('order_status') == 'invoiced', 'faturado').
                                            when(F.col('order_status') == 'delivered', 'entregue').
                                            when(F.col('order_status') == 'unavailable', 'indisponível').
                                            when(F.col('order_status') == 'created', 'criado').
                                            when(F.col('order_status') == 'approved', 'aprovado').
                                            otherwise("NÃO INFORMADO")))



In [0]:
display(pedidos_bronze_df.limit(15))

pedidos_silver_df = (pedidos_bronze_df.
                     select(
                         F.col("order_id").alias("id_pedido"),
                         F.col("customer_id").alias("id_consumidor"),
                         F.col("order_status").alias("status"),
                         F.col("order_purchase_timestamp").alias("pedido_compra_timestamp"),
                         F.col("order_approved_at").alias("pedido_aprovado_timestamp"),
                         F.col("order_delivered_carrier_date").alias("pedido_carregado_timestamp"),
                         F.col("order_delivered_customer_date").alias("pedido_entregue_timestamp"),
                         F.col("order_estimated_delivery_date").alias("pedido_estimativa_entrega_timestamp"),
                         F.col("ingestion_timestamp").alias("data_ingestao"))
                     .withColumn("tempo_entrega_dias", F.date_diff(F.col("pedido_entregue_timestamp"), F.col("pedido_compra_timestamp")))

                     .withColumn("tempo_entrega_estimado_dias", F.date_diff(F.col
                     ("pedido_estimativa_entrega_timestamp"), F.col("pedido_compra_timestamp")))

                     .withColumn("diferenca_entrega_dias", F.col("tempo_entrega_dias")- F.col("tempo_entrega_estimado_dias"))

                     .withColumn("entrega_no_prazo", 
                                 F.when(F.col("diferenca_entrega_dias") <= 0, "Sim").
                                 when(F.col("status") != "entregue", "Não entregue").
                                 otherwise("Não"))

                    .withColumn('data_ingestao', F.current_timestamp()))



In [0]:


pedidos_itens_silver_df = (
    pedidos_itens_bronze_df
    .withColumnRenamed('order_id', 'id_pedido')
    .withColumnRenamed('order_item_id', 'id_item')
    .withColumnRenamed('product_id', 'id_produto')
    .withColumnRenamed('seller_id', 'id_vendedor')
    .withColumnRenamed('product_category_name', 'categoria_produto')
    .withColumnRenamed('price', 'preco_BRL')
    .withColumnRenamed('freight_value', 'preco_frete')
    .withColumnRenamed('ingest_data', 'data_ingestao')
    .withColumn('preco_BRL', F.col('preco_BRL').cast(T.DecimalType(12,2)))
    .withColumn('preco_frete', F.col('preco_frete').cast(T.DecimalType(12,2)))
    .withColumn('data_ingestao', F.current_timestamp())
)

display(pedidos_itens_silver_df.limit(15))

In [0]:
display(pagamentos_bronze_df.limit(10))

pagamentos_bronze_df = (
    pagamentos_bronze_df
    .withColumn(
        "order_status",
        F.when(F.col("payment_type") == "credit_card", "Cartão de Crédito")
         .when(F.col("payment_type") == "boleto", "Boleto")
         .when(F.col("payment_type") == "voucher", "Voucher")
         .when(F.col("payment_type") == "debit_card", "Cartão de Débito")
         .otherwise("Outro")
    )
)

In [0]:
pagamentos_bronze_df = (
    pagamentos_bronze_df
    .select(
        F.col("order_id").alias("id_pedido"),
        F.col("payment_sequential").alias("codigo_pagamento"),
        F.col("payment_type").alias("forma_pagamento"),
        F.col("payment_installments").alias("parcelas"),
        F.col("payment_value").alias("valor_pagamento"),
        F.col("ingestion_timestamp").alias("data_ingestao"),
    )
    .withColumn('valor_pagamento', F.col('valor_pagamento').cast(T.DecimalType(12,2)))
    .withColumn('data_ingestao', F.current_timestamp())
)

display(pagamentos_bronze_df.limit(5))

In [0]:
produtos_bronze_df.printSchema()

df_produtos_silver = (produtos_bronze_df.
                      select(
                            F.col("product_id").alias("id_produto"),
                            F.col("product_category_name").alias("categoria_produto"),
                            F.col("product_lenght_cm").alias("comprimento_centimetros"),
                            F.col("product_height_cm").alias("altura_centimetros"),
                            F.col("product_width_cm").alias("largura_centimetros"),
                            F.col("product_weight_g").alias("peso_produto_gramas"),
                            F.col("ingestion_timestamp").alias("data_ingestao"),
                      )
                      .withColumn('data_ingestao', F.current_timestamp())
)

In [0]:
categoria_produtos_traducao_silver_df = (
    categoria_produtos_traducao_bronze_df
    .select(
        F.col("product_category_name").alias("nome_produto_pt"),
        F.col("product_category_name_english").alias("nome_produto_en"),
        F.col("ingestion_timestamp").alias("data_ingestao"),
    )
    .withColumn('data_ingestao', F.current_timestamp())
)

In [0]:
vendedores_silver_df = (vendedores_bronze_df
                        .select(
                            F.col("seller_id").alias("id_vendedor"),
                            F.col("seller_zip_code_prefix").alias("prefixo_cep"),
                            F.upper(F.col("seller_city")).alias("cidade"),
                            F.upper(F.col("seller_state")).alias("estado"),
                            F.col("ingestion_timestamp").alias("data_ingestao"),
                        )
                        .withColumn('data_ingestao', F.current_timestamp()))
                        

## Coluna: id_pedido
* id_pedido não pode ser **nulo**

##Colunas: data_comentario, data_resposta
* Tem que estar no formato: yyyy/mm/dd
* Não pode ser nulo

In [0]:
nulos_df = avaliacoes_pedidos_bronze_df.select([
    F.count(F.when(F.col(c).isNull(), c)).alias(c)
    for c in avaliacoes_pedidos_bronze_df.columns
]).display()

In [0]:
avaliacoes_pedidos_bronze_df.printSchema()
display(avaliacoes_pedidos_bronze_df.limit(20))

In [0]:
avaliacoes_pedidos_bronze_validados_df = (avaliacoes_pedidos_bronze_df
                                .filter(
                                    (F.col("review_creation_date").isNotNull().rlike(r'[^\\d\\/]')) |
                                    (F.col("review_answer_timestamp").isNotNull().rlike(r"^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}$"))
                                    |
                                    (F.col("order_id").isNotNull())
                                )
                                .withColumn('data_ingestao', F.current_timestamp()))

print("Quantidade de linhas removidas: ", avaliacoes_pedidos_bronze_df.count()-avaliacoes_pedidos_bronze_validados_df.count())



In [0]:
avaliacoes_pedidos_silver_df = (
    avaliacoes_pedidos_bronze_df
    .select(
        F.col("review_id").alias("id_avaliacao"),
        F.col("order_id").alias("id_pedido"),
        F.col("review_score").alias("avaliacao"),
        F.col("review_comment_title").alias("titulo_comentario"),
        F.col("review_comment_message").alias("comentario"),
        F.col("review_creation_date").alias("data_comentario"),
        F.col("review_answer_timestamp").alias("data_resposta"),
        F.col("ingestion_timestamp").alias("data_ingestao"),
    )
    .withColumn('avaliacao', F.col('avaliacao').cast(T.IntegerType()))
    .withColumn('data_resposta', F.to_date(F.col('data_resposta'), 'yyyy-MM-dd HH:mm:ss'))
    .withColumn('data_comentario', F.to_date(F.col('data_comentario'), 'yyyy-MM-dd HH:mm:ss'))
    .withColumn('data_ingestao', F.current_timestamp())
)

display(avaliacoes_pedidos_silver_df.limit(15))

In [0]:
display(avaliacoes_pedidos_bronze_df.select(["review_creation_date", "review_answer_timestamp"]).limit(30))

In [0]:
avaliacoes_pedidos_null_order_df = avaliacoes_pedidos_bronze_df.filter(F.col("order_id").isNull())
display(avaliacoes_pedidos_null_order_df.limit(20))

In [0]:
pedidos_orfaos = (pedidos_silver_df
                  .join(
                      consumidores_silver_df,
                      on="id_consumidor",
                      how="leftanti",
                  ))
print(pedidos_orfaos.count())

In [0]:
pedidos_item_orfaos = (pedidos_itens_silver_df
                       .join(
                           pedidos_silver_df,
                           on="id_pedido",
                           how="leftanti",
                       ))

print(pedidos_item_orfaos.count())

In [0]:
def remover_orfaos(df_silver, df_orfao, match):
    if (df_orfao.count() > 0):
        return df_silver
    
    return (df_silver.join(
        df_orfao,
        on=f"{match}",
        how="leftanti",
    ))

pedidos_itens_silver_df = remover_orfaos(pedidos_itens_silver_df, pedidos_item_orfaos, "id_pedido")
pedidos_silver_df = remover_orfaos(pedidos_silver_df, pedidos_orfaos, "id_conumidor")